#### Example how to read the data in MNE-python 


In [1]:
import mne
import numpy as np
import matplotlib as plt
from scipy.io import loadmat 
import pandas as pd
import seaborn as sns

%matplotlib qt
mne.set_log_level("WARNING")

In [2]:
fname_mat = 'EEG_Data/s41.mat'
Data = loadmat(fname_mat)

In [3]:
# Motor Execution
EEG_ME_Left = Data['eeg']['movement_left'][0][0]
EEG_ME_Right = Data['eeg']['movement_right'][0][0]
ME_Event = Data['eeg']['movement_event'][0][0]

In [4]:
# Motor Imagery
EEG_MI_Left = Data['eeg']['imagery_left'][0][0]
EEG_MI_Right = Data['eeg']['imagery_right'][0][0]
MI_Event = Data['eeg']['imagery_event'][0][0]

In [5]:
# Resting State
RS = Data['eeg']['rest'][0][0]

# Blinking - Noise1
Noise1 = Data['eeg']['noise'][0][0][0][0]

# eyeball up-down - Noise2
Noise2 = Data['eeg']['noise'][0][0][1][0]

# eyeball left-right - Noise3
Noise3 = Data['eeg']['noise'][0][0][2][0]

# head movement - Noise4
Noise4 = Data['eeg']['noise'][0][0][3][0]

# jaw clenching - Noise5
Noise5 = Data['eeg']['noise'][0][0][4][0]

In [6]:
#General Info
sfreq = Data['eeg']['srate'][0][0][0][0] #Sampling Frequency
psenloc = Data['eeg']['psenloc'][0][0] #Sensor location projected to unit sphere
senloc = Data['eeg']['senloc'][0][0] #Sensor locations 3D
bad_trial_indices_voltage = Data['eeg']['bad_trial_indices'][0][0][0][0][0][0][0] #Bad trials indices voltage
bad_trial_indices_mi = Data['eeg']['bad_trial_indices'][0][0][0][0][0][0][1] #Bad trials indices MI
names_types = pd.read_excel(r'ch_names&types.xlsx') #Read file with ch names and types
ch_types = names_types['ch_types'].tolist() #ch_types
ch_names = names_types['ch_names'].tolist() #ch_names


In [7]:
# Create stimuli channel
Noise1_Event = np.zeros((1,Noise1.shape[1]))
Noise1_Event[0,0] = 1
Noise2_Event = np.zeros((1,Noise2.shape[1]))
Noise2_Event[0,0] = 2
Noise3_Event = np.zeros((1,Noise3.shape[1]))
Noise3_Event[0,0] = 3
Noise4_Event = np.zeros((1,Noise4.shape[1]))
Noise4_Event[0,0] = 4
Noise5_Event = np.zeros((1,Noise5.shape[1]))
Noise5_Event[0,0] = 5
RS_Event = np.zeros((1,RS.shape[1]))
RS_Event[0,0] = 6
ME_Event_Rigth = ME_Event * 7
ME_Event_Left = ME_Event * 8
MI_Event_Rigth = MI_Event * 9
MI_Event_Left = MI_Event * 10

# Join All Events and create stimuli channels
Event_Stimuli = np.concatenate((Noise1_Event, Noise2_Event, Noise3_Event, Noise4_Event, Noise5_Event, RS_Event, ME_Event_Rigth, ME_Event_Left, MI_Event_Rigth, MI_Event_Left), axis=1)
#Event_Stimuli = np.concatenate((ME_Event_Rigth, ME_Event_Left, MI_Event_Rigth, MI_Event_Left), axis=1)

In [8]:
# Create raw data
Chs_Data = np.concatenate((Noise1, Noise2, Noise3, Noise4, Noise5, RS, EEG_ME_Right, EEG_ME_Left, EEG_MI_Right, EEG_MI_Left), axis=1) * 1e-6 #Convert from uV to V
#Chs_Data = np.concatenate((EEG_ME_Right, EEG_ME_Left, EEG_MI_Right, EEG_MI_Left), axis=1)

In [9]:
# Event ID definition
event_id = {'Blink':1,'Up/Down':2,'Left/Right':3,'Jaw':4,'HeadMov':5, 'Resting':6, 'ME_Right':7, 'ME_Left':8, 'MI_Right':9, 'MI_Left':10}
#event_id = {'ME_Right':7, 'ME_Left':8, 'MI_Right':9, 'MI_Left':10}

In [10]:
# Create Info
info = mne.create_info(ch_names, sfreq, ch_types)

In [11]:
# Create a montage based on the electrode positions
SensorLoc = dict(zip(ch_names[0:64], psenloc.tolist()))
montage = mne.channels.make_dig_montage(ch_pos=SensorLoc, coord_frame='head')

# Include the montage in the info
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 64 EEG, 4 EMG, 1 Stimulus
 custom_ref_applied: False
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 69
 projs: []
 sfreq: 512.0 Hz
>

In [12]:
# Create raw object
Raw_Data = np.concatenate((Chs_Data,Event_Stimuli), axis=0) 
raw = mne.io.RawArray(Raw_Data, info)

# Read events from Stimuli channel
events = mne.find_events(raw, stim_channel='Stim', verbose=True, initial_event=True)

246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]


In [13]:
# Plot Data
# raw.plot(events=events,event_id=event_id, lowpass=45,highpass=0.5)

In [14]:
# Only imaginary events
event_id_M = {'MI_Right':9, 'MI_Left':10}
events_M = events[events[:,2] > 8]


In [15]:
# Plot Raw Data only with Movements
# raw.plot(events=events_M,event_id=event_id_M)

In [16]:
# Take a segment and plot the psd of the raw data
# raw.copy().crop(500,800).plot_psd(fmax=100)

In [17]:
biosemi64_montage = mne.channels.make_standard_montage('biosemi64')
raw.set_montage(biosemi64_montage)

<RawArray | 69 x 919552 (1796.0 s), ~484.2 MB, data loaded>

In [18]:
# As the 2D plot looks strange, we can modifi the sphere by adding sphere='eeglab'
# raw.plot_sensors(kind='topomap',ch_type='eeg', show_names=True, sphere='eeglab')

In [19]:
#Lets visualize in 3D to see if they are ok
# raw.plot_sensors(kind='3d',ch_type='eeg', show_names=True)

In [20]:
# Create Epochs
epochs = mne.Epochs(raw, events_M, event_id_M, tmin=-1.95, tmax=4.95, picks=('eeg'), baseline=None, preload=True)
epochs.filter(0.5,45)

Number of events,200
Events,MI_Left: 100MI_Right: 100
Time range,-1.949 – 4.949 sec
Baseline,off


In [21]:
#Plot PSD of the epochs to confirm
# epochs.plot_psd(fmax=50)

#### Applying ICA to epochs

In [22]:
#Defining ICA
# ica = mne.preprocessing.ICA(n_components=40, max_iter="auto", random_state=97)

#Fit ICA to the epochs
# ica.fit(epochs)

In [23]:
# Plot a topography map of the components
# ica.plot_components()

In [24]:
# Plot the time courses of the source separation (sources in ICA does not mean brain sources)
# ica.plot_sources(epochs, show_scrollbars=False)

In [25]:
# Plot the properties of particular components
### components = [i for i in range(0, 40)]
# ica.plot_properties(epochs,[0,1,2,7])

In [26]:
# Exclude components 0 and 6
# ica.exclude = [0,1,2] # This is decided manuallyt after analyzing the components

In [27]:
#Apply ICA to the data
# ica.apply(epochs)

In [28]:
# Common Average Reference
epochs.set_eeg_reference(ref_channels='average',projection=True)

Number of events,200
Events,MI_Left: 100MI_Right: 100
Time range,-1.949 – 4.949 sec
Baseline,off


In [29]:
# Create epoch for TF analysis
channels = ['C3','C4']
epochs_TF = epochs.copy().apply_proj().pick_channels(channels)

In [30]:
# Compute TF Maps - Wavelet
freqs = np.arange(2, 35.5, 0.5)
n_cycles = freqs / 2.0  # different number of cycle per frequency
power_MI = mne.time_frequency.tfr_morlet(epochs_TF['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)


In [31]:
# Plotting TF
baseline = (-1,0)
mode = 'percent' #‘mean’ | ‘ratio’ | ‘logratio’ | ‘percent’ | ‘zscore’ | ‘zlogratio’
vmin=-1.2
vmax=1.2
#mode = 'mean'
#vmin=None
#vmax=None
fmin = 5 
fmax = 28
cmap = 'jet'
channels = ['C3','C4']
for channel in channels:
    power_MI['MI_Right'].copy().average().plot(picks=channel, baseline=baseline, mode=mode ,vmin=vmin, vmax=vmax, fmin=fmin, fmax=fmax, cmap=cmap, title=channel+ ' - Right - '+mode)
    power_MI['MI_Left'].copy().average().plot(picks=channel, baseline=baseline, mode=mode ,vmin=vmin, vmax=vmax, fmin=fmin, fmax=fmax, cmap=cmap, title=channel+ ' - Left - '+mode)

In [32]:
# Compute TF Maps - Multitaper
freqs = np.arange(2, 36)
#n_cycles = freqs / 2.0  # different number of cycle per frequency
power_MI= mne.time_frequency.tfr_multitaper(epochs_TF['MI_Left','MI_Right'], freqs=freqs, n_cycles=freqs, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)


In [33]:
# Plotting TF
baseline = (-1,0)
mode = 'percent' #‘mean’ | ‘ratio’ | ‘logratio’ | ‘percent’ | ‘zscore’ | ‘zlogratio’
vmin=-1.2
vmax=1.2
#mode = 'mean'
#vmin=None
#vmax=None
fmin = 5 
fmax = 28
cmap = 'jet'
channels = ['C3','C4']
for channel in channels:
    power_MI['MI_Right'].copy().average().plot(picks=channel, baseline=baseline, mode=mode ,vmin=vmin, vmax=vmax, fmin=fmin, fmax=fmax, cmap=cmap, title=channel+ ' - Right - '+mode)
    power_MI['MI_Left'].copy().average().plot(picks=channel, baseline=baseline, mode=mode ,vmin=vmin, vmax=vmax, fmin=fmin, fmax=fmax, cmap=cmap, title=channel+ ' - Left - '+mode)

In [34]:
# Statistic ERDs Maps

# Import function
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from matplotlib.colors import TwoSlopeNorm
import matplotlib.pyplot as plt

# Statistical parameters
vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
cnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # min, center & max ERDS

kwargs = dict(
    n_permutations=100, step_down_p=0.05, seed=1, buffer_size=None, out_type="mask"
)  # for cluster test


In [35]:
# Averaging Time-Frequency Maps
tmin, tmax = -1, 4
baseline = (-1, 0) 
tfr = power_MI.copy().crop(tmin, tmax).apply_baseline(baseline, mode="percent")

In [36]:
# Perform statistics on Time-Frequency
event_ids = ['MI_Right','MI_Left'] # events
 
for event in event_ids:
    # select desired epochs for visualization
    tfr_ev = tfr[event]
    fig, axes = plt.subplots(
        1, 3, figsize=(12, 4), gridspec_kw={"width_ratios": [10, 10, 1]}
    )
    fig.canvas.manager.set_window_title(f"ERDS map ({event})")
    for ch, ax in enumerate(axes[:-1]):  # for each channel
        # positive clusters
        _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch], tail=1, **kwargs)
        # negative clusters
        _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch], tail=-1, **kwargs)

        # note that we keep clusters with p <= 0.05 from the combined clusters
        # of two independent tests; in this example, we do not correct for
        # these two comparisons
        c = np.stack(c1 + c2, axis=2)  # combined clusters
        p = np.concatenate((p1, p2))  # combined p-values
        mask = c[..., p <= 0.05].any(axis=-1)

        # plot TFR (ERDS map with masking)
        tfr_ev.average().plot(
            [ch],
            cmap="RdBu",
            cnorm=cnorm,
            axes=ax,
            colorbar=False,
            show=False,
            mask=mask,
            mask_style="mask",
        )

        ax.set_title(tfr.ch_names[ch], fontsize=10)
        ax.axvline(0, linewidth=1, color="black", linestyle=":")  # event
        if ch != 0:
            ax.set_ylabel("")
            ax.set_yticklabels("")
    fig.colorbar(axes[0].images[-1], cax=axes[-1]).ax.set_yscale("linear")
    fig.suptitle(f"ERDS ({event})")
    plt.show()

In [41]:
# Compute and plot ERDS
df = tfr.to_data_frame(time_format=None, long_format=True)

# Map to frequency bands:
freq_bounds = {"_": 0, "delta": 4, "theta": 8, "alpha": 13, "beta": 30, "gamma": 140}
df["band"] = pd.cut(df["freq"], list(freq_bounds.values()), labels=list(freq_bounds)[1:])

# Filter to retain only relevant frequency bands:
freq_bands_of_interest = ["alpha"]
df = df[df.band.isin(freq_bands_of_interest)]
df["band"] = df["band"].cat.remove_unused_categories()

# Order channels for plotting:
df["channel"] = df["channel"].cat.reorder_categories(("C3","C4"), ordered=True)

g = sns.FacetGrid(df, row="band", col="channel", margin_titles=True)
g.map(sns.lineplot, "time", "value", "condition", n_boot=10)
axline_kw = dict(color="black", linestyle="dashed", linewidth=0.5, alpha=0.5)
g.map(plt.axhline, y=0, **axline_kw)
g.map(plt.axvline, x=0, **axline_kw)
#g.set(ylim=(None, 1.5))
g.set_axis_labels("Time (s)", "ERDS (%)")
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.add_legend(ncol=2, loc='upper center')
g.fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.08)


Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...


###### Coments

In [38]:
# Check this for importing the current electrode montage: https://mne.tools/stable/generated/mne.channels.make_dig_montage.html#mne.channels.make_dig_montage
#mne.channels.make_dig_montage(ch_pos=senloc, coord_frame='head')
